In [ ]:
import json
import matplotlib.pyplot as plt
from matplotlib.pyplot import plot,savefig
import numpy as np
import pandas as pd
import copy
import collections
import matplotlib.image as mpimg
from scipy import misc
import sklearn
from sklearn import linear_model
from nltk.text import TextCollection
from nltk.stem import PorterStemmer 
import csv


In [ ]:
cd C:\Users\DH\Desktop\M3test

In [ ]:
#Read data
list_bus = []
    
with open('list_bus.json','r') as load_f:
    list_bus = json.load(load_f)
    
list_rev = []
    
with open('review_final.json','r') as load_f:
    list_rev = json.load(load_f)

In [ ]:
#Create a new dataset called businessPlusRev,which includes: 'business_id', 'business_name',
# 'business_star', 'review_text' and 'review_star'
businessPlusRev=[]
business_id=[]
for i in range(len(list_bus)):
    business_id.append(list_bus[i]['business_id'])

for i in range(len(business_id)):
    bus_id=business_id[i]
    for j in range(len(list_bus)):
        if list_bus[j]['business_id'] == bus_id:
            bus_name=list_bus[j]['name']
            bus_star=float(list_bus[j]['stars'])
    rev_text=[]
    rev_star=[]
    for k in range(len(list_rev)):
        if list_rev[k]['business_id']==bus_id:
            rev_text.append(list_rev[k]['text'])
            rev_star.append(float(list_rev[k]['stars']))
    pair=dict()
    pair['business_id']=bus_id
    pair['business_name']=bus_name
    pair['business_star']=bus_star
    pair['review_text']=rev_text
    pair['review_star']=rev_star
    businessPlusRev.append(pair)
    
  

In [ ]:
#Function to get all the words in all of each business' reviews    
def getWords(x):
    words=[]
    n=len(x['review_text'])
    for i in range(n):
        text=x['review_text'][i]
        for j in range(len(text)):
            if text[j] not in words:
                words.append(text[j])
                
    return(words)
    

In [ ]:
#Function to transform each business' review to matrix 
#according to the count of each word in all reviews
def getDataFrame(x):
    dataFrame_dict=dict()
    words=getWords(x)
    n=len(x['review_text'])   
    for i in range(len(words)):
        num=n*[0]
        eachWord=words[i]
        for j in range(n):
            text=x['review_text'][j]
            for k in range(len(text)):
                if text[k]==eachWord:
                    num[j]+=1
        dataFrame_dict[eachWord]=num
    
    dataFrame=pd.DataFrame(dataFrame_dict)
    return(dataFrame)


In [ ]:
#Function to get each business' top rank words
def makeModel(x):
    words=getWords(x)
    X=np.array(getDataFrame(x))
    Y=np.array(x['review_star'])
    reg = linear_model.LinearRegression().fit(X, Y)
    
    word_dict=dict()
    for i in range(len(words)):
        word_dict[words[i]]=reg.coef_[i]
    word_sort=sorted(word_dict.items(), key=lambda e:e[1], reverse=True)
    
    name=x['business_name']
    if len(word_sort)>=20:
        word_pos=word_sort[:10]
        word_neg=word_sort[-10:]
    else:
        word_pos=word_sort[:round(len(word_sort)/2)]
        word_neg=word_sort[-round(len(word_sort)/2):]    
    wordMaxMin=[]
    wordMaxMin.append(word_pos)
    wordMaxMin.append(word_neg)
    #rank=dict()
    #rank[name]=wordMaxMin
    return(name,wordMaxMin)

In [ ]:
#Get all the model coefficients of business
model=dict()
for i in range(len(businessPlusRev)):
    x=businessPlusRev[i]
    (name,rank)=makeModel(x)
    model[name]=rank

In [ ]:
##Function to make advice according to the business' name   
raw_rev = []    
with open('raw_rev.json','r') as load_f:
    raw_rev = json.load(load_f)    
        
def makeRawAdvice(x):
    rank=model[x]
    pos=rank[0]
    neg=rank[1]
    pos_word=[]
    neg_word=[]
    for i in range(len(pos)):
        pos_word.append(pos[i][0])
    for i in range(len(neg)):
        neg_word.append(neg[len(neg)-i-1][0])
    for i in range(len(pos_word)):
        if pos_word[i].endswith('_neg'):
            trans=pos_word[i].split('_')[0]
            pos_word[i]=trans
    for i in range(len(pos_word)):
        if neg_word[i].endswith('_neg'):
            trans=neg_word[i].split('_')[0]
            neg_word[i]=trans

    testPos=pos_word[0]
    testNeg=neg_word[0]
    #print(testPos)
    
    for i in range(len(businessPlusRev)):
        if businessPlusRev[i]['business_name']==x:
            business_id=businessPlusRev[i]['business_id']
            break
    
    advice_pos=[]
    advice_neg=[]
    for i in range(len(raw_rev)):
        if raw_rev[i]['business_id']==business_id:
            text=raw_rev[i]['text']
            for j in range(len(pos_word)):
                Pos=pos_word[j]
                if text.find(Pos,0, len(text))!=-1:
                    index=text.find(Pos,0,len(text))
                    begin1=text[0:index].rfind('.')
                    begin2=text[0:index].rfind('!')
                    begin=begin1 if begin1>begin2 else begin2
                    end1=text[index:len(text)].find('.')
                    end2=text[index:len(text)].find('!')
                    end=end1 if end1<end2 else end2
                    textFind=text[begin+1:index+end+1]
                    if textFind not in advice_pos:
                        if len(textFind)>30:                            
                            advice_pos.append(textFind)
            for k in range(len(pos_word)):
                Neg=neg_word[k]
                if text.find(Neg,0, len(text))!=-1:
                    index=text.find(Neg,0,len(text))
                    begin1=text[0:index].rfind('.')
                    begin2=text[0:index].rfind('!')
                    begin=begin1 if begin1>begin2 else begin2
                    end1=text[index:len(text)].find('.')
                    end2=text[index:len(text)].find('!')
                    end=end1 if end1<end2 else end2

                    textFind=text[begin+1:index+end+1]
                    if textFind not in advice_neg:
                        if len(textFind)>30:
                            advice_neg.append(textFind)
    if len(advice_pos)>=3:
        advice_pos=advice_pos[0:3]
    if len(advice_neg)>=3:
        advice_neg=advice_neg[0:3]
    for i in range(len(advice_pos)):
        advice_pos[i]=advice_pos[i].replace('\n','')
        advice_pos[i]=advice_pos[i].replace('\r','')
    for i in range(len(advice_neg)):
        advice_neg[i]=advice_neg[i].replace('\n','')
        advice_neg[i]=advice_neg[i].replace('\r','')
    advice="These sentences will affect the increase of business stars: "+'\n'.join(advice_pos)+"\nThese sentences will affect the decrease of business stars:\n"+'\n'.join(advice_neg)+'\n'
    return(advice)

In [ ]:
#Add review advice to business' attributes advice
file=pd.read_table('business_advice.txt')
for i in range(len(file['Business_name'])):
    name=file['Business_name'][i]
    advice=makeAdvice(name)
    file['advice'][i]=file['advice'][i]+advice
file.to_csv('result.csv')

name=file['Business_name'][0]
advice=makeAdvice(name)
file['advice'][0]+'\n'+advice

In [ ]:
#Get csv of business' advice according to their reviews
reviewAdvice=[]
for i in range(len(businessPlusRev)):
    name=businessPlusRev[i]['business_name']
    advice=makeAdvice(name)
    x=dict()
    x['Business_name']=name
    x['advice']=advice    
    reviewAdvice.append(x)


 
headers = ['Business_name', 'advice'] 
datas = reviewAdvice
 
filename = 'test.csv'
with open(filename, 'w', encoding='utf-8',newline='') as f:
    
    writer = csv.DictWriter(f, headers)
    writer.writeheader()
    
    for row in datas:
        writer.writerow(row)

In [ ]:
#Get csv of the full contents for the webpage
        
full=[]      
        
file1=pd.read_table('business_advice.txt')
file2=pd.read_csv('rawReview.csv')
file3=pd.read_csv('id_page.csv')

for i in range(len(file1['Business_name'])):
    bussiness_name=file1['Business_name'][i]
    advice1=file1['advice'][i]
    zipcode=int(file1['zipcode'][i])
    city=file1['city'][i]
    stars=file1['stars'][i]
    for j in range(len(file2['Business_name'])):
        if file2['Business_name'][j]==bussiness_name:
            advice2=file2['advice'][j]
            break
    id=file3['id'][i]
    page=file3['page'][i]
    for j in range(len(businessPlusRev)):
        if businessPlusRev[j]['business_name']==bussiness_name:
            rev_stars=businessPlusRev[j]['review_star']
            count_num=[0,0,0,0,0]
            for k in range(len(rev_stars)):
                point=float(rev_stars[k])
                if point <= 1:
                    count_num[0] += 1
                elif point <= 2:
                    count_num[1] += 1
                elif point <= 3:
                    count_num[2] += 1
                elif point <= 4:
                    count_num[3] += 1
                elif point <= 5:
                    count_num[4] += 1
            
            star_1=count_num[0]
            star_2=count_num[1]
            star_3=count_num[2]
            star_4=count_num[3]
            star_5=count_num[4]
            break
    
    x=dict()
    x['bussiness_name']=bussiness_name
    x['advice1']=advice1
    x['advice2']=advice2
    x['zipcode']=zipcode
    x['city']=city
    x['id']=id
    x['page']=page
    x['1_star']=star_1
    x['2_star']=star_2
    x['3_star']=star_3
    x['4_star']=star_4
    x['5_star']=star_5
    x['stars']=stars
    
    
    full.append(x)


headers = ['bussiness_name','advice1','advice2','zipcode','city','id','page','1_star','2_star','3_star','4_star','5_star','stars'] 
datas = full
 
filename = 'full.csv'
with open(filename, 'w', encoding='utf-8',newline='') as f:
    
    writer = csv.DictWriter(f, headers)
    writer.writeheader()
    
    for row in datas:
        writer.writerow(row) 
